In [1]:
#!/usr/bin/python
# updated DBR 04/2023 #

import numpy as np
import pandas as pd
import os
import scipy.optimize as opt #for power law fitting
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings(action='once')


In [8]:
#function that cleans up TCR data and does the unions/intersections etc
#this doesn't do any resampling yet

def make_good_df(fn,restrict_persistent_TCR):

    pid=fn.split('.')[0] #get the participant full id
    spid=pid.split('_')[0] #get the participant short id, no timepoint
    pid_tp='t0'
    if '_' in pid:
        pid_tp = pid.split('_')[1]

    #deal with some longitudinal, keep only persistent among t1 and t2s
    if pid_tp=='t0':
        df0 = pd.read_csv('rawTCRdata/'+fn, sep='\t',
                         usecols=['extended_rearrangement','cdr3_amino_acid','templates','frame_type'],low_memory=False).dropna()
        good_df=df0[df0['frame_type']=='In'] #restrict to only in frame rearrangements

    else:
        t0fn = pid.split('_')[0]+'_t0.tsv' #get the zero fn

        #read the current timepoint ti
        dfi = pd.read_csv('rawTCRdata/'+fn, sep='\t',
                          usecols=['extended_rearrangement','cdr3_amino_acid','templates','frame_type'],low_memory=False).dropna()        
        dfi=dfi[dfi['frame_type']=='In'] #restrict to only in frame rearrangements

        if restrict_persistent_TCR:
            #also read the zero fn
            df0 = pd.read_csv('rawTCRdata/'+t0fn, sep='\t',
                         usecols=['extended_rearrangement','cdr3_amino_acid','templates','frame_type'],low_memory=False).dropna()
            df0=df0[df0['frame_type']=='In'] #restrict to only in frame rearrangements

            union_df = pd.merge(df0, dfi, how='outer', on=['extended_rearrangement']) #the union of all TCR
            union_df = union_df.fillna(0) #fill all zeros

            persistent_dfi = union_df[union_df['templates_x']>0] #restrict to TCR found at t0
            good_df = persistent_dfi[['extended_rearrangement','templates_y','frame_type_y']]  
            good_df = good_df.rename(columns={'templates_y':'templates'})
            good_df = good_df[good_df['frame_type_y']=='In']
        else: #if not restricted
            good_df=dfi
            
    return pid,good_df


In [3]:
#loop through all ppts, print clean dataframes to csv using function above

#get all filenames in this folder

fns = os.listdir('rawTCRdata/')
fns.remove('.DS_Store') #drop hidden file
fns.remove('excluded') #drop this folder
fns.sort()

Nl=[]
for i,fn in enumerate(fns):
    pid,good_df = make_good_df(fn,restrict_persistent_TCR=False)
    good_df[['extended_rearrangement','cdr3_amino_acid','templates']].to_csv('cleanTCRdata/'+pid+'.csv') #output the clean data, note this hasn't been downsampled yet

    Nl.append(np.sum(good_df['templates'])) #keep track of original N
    
#print(np.gmean(Nl))

In [4]:
print(np.mean(np.log10(Nl)),np.std(np.log10(Nl)))

4.926304672482832 0.2690675429776805


In [9]:
#for just the multi-time participants, make "restricted" data frames
Nlr=[]

#not using PWH122 and MACS14218
PWH_ppts = ['PWH22','PWH548','PWH583','PWH746']
ctl_ppts = ['MACS10136','MACS13042','MACS14173','WIHS']

for ppt in PWH_ppts+ctl_ppts:
    for tp in [1,2]:
        fn=ppt+'_t'+str(tp)+'.tsv'
        pid,good_df = make_good_df(fn,restrict_persistent_TCR=True)
        good_df[['extended_rearrangement','templates']].to_csv('cleanTCRdata/restricted/'+pid+'.csv') #output the clean data, note this hasn't been downsampled yet
        Nlr.append(np.sum(good_df['templates'])) #keep track of original N



In [10]:
print(np.mean(np.log10(Nlr)),np.std(np.log10(Nlr)))

4.447572135444368 0.4071273484492284
